<a href="https://colab.research.google.com/github/asdfasdf001234/2024-1-MLPRJ/blob/main/Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
import os
import time
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [121]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import seaborn as sns

In [122]:
# pretrained 관련
import torch
import torchvision.transforms as v2
from torchvision import models

In [123]:
import torch
import numpy as np
import random

def set_random_seed(seed_value):
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)

# Set a random seed value
seed_value = 42
set_random_seed(seed_value)


# data 준비
- personal color data (train/val/test)
- data: 1-봄 웜 / 2-여름 쿨 / 3-가을 웜 / 4-겨울 쿨
- labeling: 0-봄 웜 / 1-여름 쿨 / 2-가을 웜 / 3-겨울 쿨

In [124]:
from torch.utils.data import Dataset , DataLoader
from torchvision import datasets
from torchvision.transforms import v2

In [125]:
from PIL import Image
from glob import glob

In [126]:
data_path = "./drive/MyDrive/Colab Notebooks/MLPRJ/img_model_test"

In [127]:
import pandas as pd
import os
from glob import glob

def create_dataframe(data_path, label_list, data_type):
    df = pd.DataFrame({"path": [], "label": [], "class_id": []})
    img_list = glob(os.path.join(data_path, '*.jpg'))

    for img in img_list:
      file_name = os.path.splitext(os.path.basename(img))[0]
      label_index = int(file_name[0]) - 1  #진영님 코드와 동일
      if 0 <= label_index < len(label_list):
        label = label_list[label_index]
        new_data = pd.DataFrame({"path": [img], "label": [label], "class_id": [label_index]})
        df = pd.concat([df, new_data], ignore_index=True)

    df[["path"]] = df[["path"]].astype(str)
    df[["label"]] = df[["label"]].astype(str)
    df[["class_id"]] = df[["class_id"]].astype(int)

    return df

In [128]:
# Example usage:
train_path = data_path + '/train'
valid_path = data_path + '/val'
test_path = data_path + '/test'
label_list = ['spring', 'summer', 'autumn', 'winter']

train_df = create_dataframe(train_path, label_list, 'training')
val_df = create_dataframe(valid_path, label_list, 'validation')
test_df = create_dataframe(test_path, label_list, 'testing')

In [129]:
print(f"train_data: {len(train_df)}")
print(f"val_data:{len(val_df)}")
print(f"test_data:{len(test_df)}")

train_data: 40
val_data:20
test_data:20


In [130]:
#데이터 출력
print("Train DataFrame:")
print(train_df.head())
print("Validation DataFrame:")
print(val_df.head())
print("Test DataFrame:")
print(test_df.head())

Train DataFrame:
                                                path   label  class_id
0  ./drive/MyDrive/Colab Notebooks/MLPRJ/img_mode...  autumn         2
1  ./drive/MyDrive/Colab Notebooks/MLPRJ/img_mode...  autumn         2
2  ./drive/MyDrive/Colab Notebooks/MLPRJ/img_mode...  autumn         2
3  ./drive/MyDrive/Colab Notebooks/MLPRJ/img_mode...  autumn         2
4  ./drive/MyDrive/Colab Notebooks/MLPRJ/img_mode...  winter         3
Validation DataFrame:
                                                path   label  class_id
0  ./drive/MyDrive/Colab Notebooks/MLPRJ/img_mode...  summer         1
1  ./drive/MyDrive/Colab Notebooks/MLPRJ/img_mode...  spring         0
2  ./drive/MyDrive/Colab Notebooks/MLPRJ/img_mode...  autumn         2
3  ./drive/MyDrive/Colab Notebooks/MLPRJ/img_mode...  winter         3
4  ./drive/MyDrive/Colab Notebooks/MLPRJ/img_mode...  autumn         2
Test DataFrame:
                                                path   label  class_id
0  ./drive/MyDrive/Col

In [131]:
class BaseDataset(torch.utils.data.Dataset):
    def __init__(self , dataframe , transforms_):
        self.df = dataframe
        self.transforms_ = transforms_

    def __len__(self):
        return len(self.df)

    def __getitem__(self ,index):
        img_path = self.df.iloc[index]['path']
        img = Image.open(img_path).convert("RGB")
        transformed_img = self.transforms_(img)
        class_id = self.df.iloc[index]['class_id']
        return transformed_img , class_id

In [132]:
Transforms = v2.Compose([
    v2.Resize((224,224)),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [133]:
BATCH_SIZE = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [134]:
#get dataloader

train_dataset = BaseDataset(train_df, Transforms) # train_transforms
val_dataset = BaseDataset(val_df, Transforms)
test_dataset = BaseDataset(test_df, Transforms)
train_loader = DataLoader(train_dataset , batch_size=BATCH_SIZE , shuffle = True)
val_loader = DataLoader(val_dataset , batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset , batch_size=BATCH_SIZE)

In [135]:
resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

In [136]:
#print(resnet50)

In [137]:
import torch.nn as nn
class MyRes50(nn.Module):
    def __init__(self, pretrained_model):
        super(MyRes50, self).__init__()
        self.backbone = pretrained_model

        for param in self.backbone.parameters():
          param.requires_grad = False

        self.dropout = nn.Dropout(0,5)
        self.extra_layer = nn.Linear(1000, 4)

    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        x = self.extra_layer(x)
        return x


In [138]:
myres50 = MyRes50(resnet50)

In [139]:
myres50.cuda()

MyRes50(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
         

In [140]:
seed_value = 42
set_random_seed(seed_value)

EPOCHS = 100
logs = {"train_loss":[] , "train_acc":[] , "val_loss":[] , "val_acc":[]}

if os.path.exists('checkpoints') == False:
    os.mkdir('checkpoints')

criterion = nn.CrossEntropyLoss()


optimizer = optim.Adam(resnet50.parameters(), lr=0.01)
Cosine_lr_scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr = 0.00001, max_lr = 0.1)


patience = 10
counter = 0
best_loss = np.inf

In [141]:
def train(dataloader , model , loss_fn , optimizer , lr_scheduler):
    size = 0
    num_batches = len(dataloader)

    model.train()
    epoch_loss , epoch_correct = 0 , 0

    for i ,(data_ , target_) in enumerate(dataloader):

        data = data_.to(device)
        target = target_.to(device)

        pred = model(data)
        _, pred_max = torch.max(pred, 1)

        loss = loss_fn(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_correct += (pred_max == target).type(torch.float).sum().item()
        size += target.size(0)


    train_acc = epoch_correct/size
    lr_scheduler.step()

    return train_acc , epoch_loss / num_batches


def test(dataloader , model , loss_fn):
    size = 0
    num_baches = len(dataloader)
    epoch_loss , epoch_correct= 0 ,0
    with torch.no_grad(): # grad 연산 X
        model.eval() # evaluation dropout 연산시
        for i, (data_ , target_) in enumerate(dataloader):

            data = data_.to(device)
            target = target_.to(device)

            pred = model(data)
            _, pred_max = torch.max(pred, 1)

            loss = loss_fn(pred, target)

            epoch_loss += loss.item()
            epoch_correct += (pred_max == target).type(torch.float).sum().item()
            size += target.size(0)

    test_acc = epoch_correct/size

    return test_acc  , epoch_loss / num_baches

In [142]:
for epoch in tqdm(range(EPOCHS)):
    train_acc , train_loss = train(train_loader ,
                                   myres50 ,
                                   criterion ,
                                   optimizer ,
                                   Cosine_lr_scheduler)

    val_acc , val_loss = test(val_loader , myres50 , criterion)
    print(f'epoch:{epoch} \
    train_loss = {train_loss:.4f} , train_acc:{train_acc:.4f} \
    val_loss = {val_loss:.4f} , val_acc:{val_acc:.4f} \
    learning rate: {optimizer.param_groups[0]["lr"]}')
    logs['train_loss'].append(train_loss)
    logs['train_acc'].append(train_acc)
    logs['val_loss'].append(val_loss)
    logs['val_acc'].append(val_acc)


    if val_loss < best_loss:
        counter = 0
        best_loss = val_loss
        #torch.save(myres50.state_dict() , "checkpoints/myvgg_16_best.pth")
    else:
        counter+=1


    if counter >= patience:
        test_acc , val_loss = test(test_loader , myres50 , criterion)
        print("Early stop !")
        print(test_acc)
        break



  1%|          | 1/100 [00:00<01:05,  1.51it/s]

epoch:0     train_loss = 1.4163 , train_acc:0.2250     val_loss = 1.4109 , val_acc:0.2500     learning rate: 5.99950000000167e-05


  2%|▏         | 2/100 [00:01<00:59,  1.65it/s]

epoch:1     train_loss = 1.4268 , train_acc:0.2750     val_loss = 1.3801 , val_acc:0.2500     learning rate: 0.00010998999999998899


  3%|▎         | 3/100 [00:01<00:57,  1.70it/s]

...

epoch:10     train_loss = 1.4297 , train_acc:0.2250     val_loss = 1.4494 , val_acc:0.2500     learning rate: 0.0005599450000000061


 12%|█▏        | 12/100 [00:06<00:43,  2.01it/s]

epoch:11     train_loss = 1.4391 , train_acc:0.1500     val_loss = 1.4521 , val_acc:0.3000     learning rate: 0.0006099399999999784


 12%|█▏        | 12/100 [00:07<00:52,  1.69it/s]

epoch:12     train_loss = 1.4200 , train_acc:0.2250     val_loss = 1.4564 , val_acc:0.3000     learning rate: 0.0006599349999999951
Early stop !
0.05
